In [1]:
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import theano.tensor as tt

In [2]:
data = bmb.load_data("sleepstudy")

In [4]:
model = bmb.Model("Reaction ~ Days + (Days | Subject)", data)
model

Formula: Reaction ~ Days + (Days | Subject)
Family name: Gaussian
Link: identity
Observations: 180
Priors:
  Common-level effects
    Intercept ~ Normal(mu: 298.5079, sigma: 261.0092)
    Days ~ Normal(mu: 0.0, sigma: 48.8915)

  Group-level effects
    1|Subject ~ Normal(mu: 0, sigma: HalfNormal(sigma: 261.0092))
    Days|Subject ~ Normal(mu: 0, sigma: HalfNormal(sigma: 48.8915))

  Auxiliary parameters
    sigma ~ HalfStudentT(nu: 4, sigma: 56.1721)


In [5]:
X = model._design.common.design_matrix
Z = model._design.group.design_matrix

In [8]:
subject_idx, subjects = pd.factorize(data["Subject"])
coords = {"subjects": list(subjects)}

with pm.Model(coords=coords) as model_pm:
    intercept_mu = pm.Normal("intercept_mu", mu=0, sigma=250)
    intercept_sigma = pm.HalfNormal("intercept_sigma", sigma=200)
    intercept_offset = pm.Normal("intercept_offset", dims="subjects")
    intercept = pm.Deterministic("intercept", intercept_mu + intercept_sigma * intercept_offset)

    days_mu = pm.Normal("days_mu", mu=0, sigma=50)
    days_sigma = pm.HalfNormal("days_sigma", sigma=50)
    days_offset = pm.Normal("days_offset", dims="subjects")
    days = pm.Deterministic("days", days_mu + days_sigma * days_offset)

    sigma = pm.HalfStudentT("sigma", nu=4, sigma=50)
    
    mu = intercept[subject_idx] + days[subject_idx] * data.Days.values
    mu = pm.Deterministic("mu", mu)
    reaction = pm.Normal("reaction", mu=mu, sigma=sigma, observed=data.Reaction.values) 

    idata_pm = pm.sample(return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma, days_offset, days_sigma, days_mu, intercept_offset, intercept_sigma, intercept_mu]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 12 seconds.


In [9]:
idata_pm.posterior.sampling_time

12.461889028549194

In [10]:
from scipy.sparse import csc_matrix
import theano

In [34]:
Z.shape

(180, 36)

In [32]:
subject_idx, subjects = pd.factorize(data["Subject"])
coords = {"subjects": list(subjects)}

with pm.Model(coords=coords) as model_pm:
    intercept = pm.Normal("intercept", mu=300, sigma=250, shape=1)
    days = pm.Normal("days", mu=0, sigma=50, shape=1)
    
    b = tt.concatenate([intercept, days])

    intercept_sigma = pm.HalfNormal("intercept_sigma", sigma=200)
    intercept_offset = pm.Normal("intercept_offset", dims="subjects")
    intercept_random = pm.Deterministic("intercept_random", intercept_sigma * intercept_offset)

    days_sigma = pm.HalfNormal("days_sigma", sigma=50)
    days_offset = pm.Normal("days_offset", dims="subjects")
    days_random = pm.Deterministic("days_random", days_sigma * days_offset)

    u = tt.concatenate([intercept_random, days_random])[:, None]
    mu = pm.math.dot(X, b) + theano.sparse.structured_dot(csc_matrix(Z), u).flatten()

    mu = pm.Deterministic("mu", mu)
    
    sigma = pm.HalfStudentT("sigma", nu=4, sigma=50)
    reaction = pm.Normal("reaction", mu=mu, sigma=sigma, observed=data.Reaction.values) 

    idata_pm = pm.sample(return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma, days_offset, days_sigma, intercept_offset, intercept_sigma, days, intercept]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 13 seconds.


In [33]:
az.summary(idata_pm)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
intercept[0],251.544,7.569,236.534,265.398,0.228,0.161,1102.0,1172.0,1.0
days[0],10.540,1.741,7.043,13.623,0.060,0.043,849.0,948.0,1.0
intercept_offset[308],0.039,0.538,-0.943,1.036,0.012,0.011,2074.0,1787.0,1.0
intercept_offset[309],-1.526,0.554,-2.589,-0.490,0.013,0.009,1883.0,1432.0,1.0
intercept_offset[310],-1.484,0.552,-2.478,-0.445,0.012,0.009,2226.0,1690.0,1.0
...,...,...,...,...,...,...,...,...,...
mu[176],334.274,9.029,317.388,351.326,0.193,0.136,2202.0,1284.0,1.0
mu[177],346.034,10.438,325.078,364.694,0.220,0.155,2261.0,1441.0,1.0
mu[178],357.795,12.147,335.762,381.165,0.251,0.178,2335.0,1476.0,1.0
mu[179],369.555,14.047,343.723,396.071,0.292,0.207,2306.0,1541.0,1.0
